<a href="https://colab.research.google.com/github/merucode/RL/blob/32-Study-Udemy-Policy_gradient/1_REINFORCE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# REINFORCE


In [ ]:
!apt-get install -y xvfb

!pip install \
  pygame \
  gym==0.23.1 \
  pytorch-lightning==1.6 \
  pyvirtualdisplay

#### Setup virtual display

In [ ]:
from pyvirtualdisplay import Display
Display(visible=False, size=(1400, 900)).start()

#### Import the necessary code libraries

In [ ]:
import copy
import torch
import random
import gym
import matplotlib

import numpy as np
import matplotlib.pyplot as plt

import torch.nn.functional as F

from collections import deque, namedtuple
from IPython.display import HTML
from base64 import b64encode

from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import IterableDataset
from torch.optim import AdamW

from pytorch_lightning import LightningModule, Trainer

from gym.wrappers import RecordVideo, RecordEpisodeStatistics, \
  NormalizeObservation, NormalizeReward


device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
num_gpus = torch.cuda.device_count()

In [ ]:
def plot_policy(policy):
  pos = np.linspace(-4.8, 4.8, 100)
  vel = np.random.random(size=(10000, 1)) * 0.1
  ang = np.linspace(-0.418, 0.418, 100)
  ang_vel = np.random.random(size=(10000, 1)) * 0.1

  g1, g2 = np.meshgrid(pos, ang)
  grid = np.stack((g1,g2), axis=-1)
  grid = grid.reshape(-1, 2)
  grid = np.hstack((grid, vel, ang_vel))

  probs = policy(grid).detach().numpy()
  probs_left = probs[:, 0]

  probs_left = probs_left.reshape(100, 100)
  probs_left = np.flip(probs_left, axis=1)

  plt.figure(figsize=(8, 8))
  plt.imshow(probs_left, cmap='coolwarm')
  plt.colorbar()
  plt.clim(0, 1)
  plt.title("P(left | s)", size=20)
  plt.xlabel("Cart Position", size=14)
  plt.ylabel("Pole angle", size=14)
  plt.xticks(ticks=[0, 50, 100], labels=['-4.8', '0', '4.8'])
  plt.yticks(ticks=[100, 50, 0], labels=['-0.418', '0', '0.418'])


def test_env(env_name, policy, obs_rms):
  env = gym.make(env_name)
  env = RecordVideo(env, 'videos', episode_trigger=lambda e: True)
  env = NormalizeObservation(env)
  env.obs_rms = obs_rms

  for episode in range(10):
    done = False
    obs = env.reset()
    while not done:
      action = policy(obs).multinomial(1).cpu().item()
      obs, _, done, _ = env.step(action)
  del env


def display_video(episode=0):
  video_file = open(f'/content/videos/rl-video-episode-{episode}.mp4', "r+b").read()
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"<video width=600 controls><source src='{video_url}'></video>")

#### Create the policy

In [ ]:
class GradientPolicy(nn.Module):

  def __init__(self, in_features, n_actions, hidden_size=128):
    super().__init__()
    self.fc1 = nn.Linear(in_features, hidden_size)
    self.fc2 = nn.Linear(hidden_size, hidden_size)
    self.fc3 = nn.Linear(hidden_size, n_actions)

  def forward(self, x):
    x = torch.tensor(x).float().to(device)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.softmax(self.fc3(x), dim=-1)
    return x

#### Plot the untrained policy

In [ ]:
policy = GradientPolicy(4, 2)
grid = plot_policy(policy)

In [ ]:
grid

#### Create the environment

In [ ]:
env = gym.vector.make("CartPole-v1", num_envs=2)

In [ ]:
env.reset()

In [ ]:
env.observation_space, env.action_space

In [ ]:
actions = np.array([0, 0])
next_obs, rewards, dones, infos = env.step(actions)

In [ ]:
next_obs

In [ ]:
rewards

In [ ]:
dones

In [ ]:
infos

In [ ]:
def create_env(env_name, num_envs):
  env = gym.vector.make(env_name, num_envs=num_envs)
  env = RecordEpisodeStatistics(env)
  env = NormalizeObservation(env)
  env = NormalizeReward(env)
  return env

#### Create the dataset

In [ ]:
class RLDataset(IterableDataset):

  def __init__(self, env, policy, steps_per_epoch, gamma):
    self.env = env
    self.policy = policy
    self.steps_per_epoch = steps_per_epoch
    self.gamma = gamma
    self.obs = env.reset()

  @torch.no_grad()
  def __iter__(self):
    transitions = []

    for step in range(self.steps_per_epoch):
      action = self.policy(self.obs)
      action = action.multinomial(1).cpu().numpy()
      next_obs, reward, done, info = self.env.step(action.flatten())
      transitions.append((self.obs, action, reward, done))
      self.obs = next_obs

    obs_b, action_b, reward_b, done_b = map(np.stack, zip(*transitions))

    running_return = np.zeros(self.env.num_envs, dtype=np.float32)
    return_b = np.zeros_like(reward_b)

    for row in range(self.steps_per_epoch - 1, -1, -1):
      running_return = reward_b[row] + (1 - done_b[row]) * self.gamma * running_return
      return_b[row] = running_return

    num_samples = self.env.num_envs * self.steps_per_epoch
    obs_b = obs_b.reshape(num_samples, -1)
    action_b = action_b.reshape(num_samples, -1)
    return_b = return_b.reshape(num_samples, -1)

    idx = list(range(num_samples))
    random.shuffle(idx)

    for i in idx:
      yield obs_b[i], action_b[i], return_b[i]

#### Create the REINFORCE algorithm

In [ ]:
class Reinforce(LightningModule):

  def __init__(self, env_name, num_envs=8, samples_per_epoch=1000, batch_size=1024,
               hidden_size=64, policy_lr=0.001, gamma=0.99, entropy_coef=0.001, optim=AdamW):

    super().__init__()

    self.env = create_env(env_name, num_envs=num_envs)

    obs_size = self.env.single_observation_space.shape[0]
    n_actions = self.env.single_action_space.n

    self.policy = GradientPolicy(obs_size, n_actions, hidden_size)
    self.dataset = RLDataset(self.env, self.policy, samples_per_epoch, gamma)

    self.save_hyperparameters()

  # Configure optimizers.
  def configure_optimizers(self):
    return self.hparams.optim(self.policy.parameters(), lr=self.hparams.policy_lr)

  def train_dataloader(self):
    return DataLoader(dataset=self.dataset, batch_size=self.hparams.batch_size)

  # Training step.
  def training_step(self, batch, batch_idx):
    obs, actions, returns = batch

    probs = self.policy(obs)
    log_probs = torch.log(probs + 1e-6)
    action_log_prob = log_probs.gather(1, actions)

    entropy = - torch.sum(probs * log_probs, dim=-1, keepdim=True)

    pg_loss = - action_log_prob * returns
    loss = (pg_loss - self.hparams.entropy_coef * entropy).mean()

    self.log("episode/PG Loss", pg_loss.mean())
    self.log("episode/Entropy", entropy.mean())

    return loss

  def training_epoch_end(self, training_step_outputs):
    self.log("episode/Return", self.env.return_queue[-1])

#### Purge logs and run the visualization tool (Tensorboard)

In [ ]:
!rm -r /content/lightning_logs/
!rm -r /content/videos/
%load_ext tensorboard
%tensorboard --logdir /content/lightning_logs/

#### Train the policy

In [ ]:
algo = Reinforce('CartPole-v1')

trainer = Trainer(
  gpus=num_gpus,
  max_epochs=100,
  log_every_n_steps=1
)

trainer.fit(algo)

#### Check the resulting policy

In [ ]:
import warnings
warnings.filterwarnings('ignore')

test_env('CartPole-v1', algo.policy, algo.env.obs_rms)

In [ ]:
display_video(episode=1)

#### Plot the trained policy

In [ ]:
plot_policy(algo.policy)